# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# OpenWeatherMap and Google developer API key
from api_keys import g_key, weather_api_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
filepath = "../WeatherPy/weather_output.csv"
city_weather = pd.read_csv(filepath)
city_weather

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,grindavik,63.84,-22.43,3.00,86,75,3.60,IS,1600840995
1,mataura,-46.19,168.86,15.56,66,66,2.68,NZ,1600841423
2,drumnadrochit,57.33,-4.48,6.11,93,75,2.10,GB,1600841424
3,busselton,-33.65,115.33,22.22,39,24,1.77,AU,1600841424
4,jamestown,42.10,-79.24,10.56,87,1,2.27,US,1600841211
...,...,...,...,...,...,...,...,...,...
463,moi,58.46,6.55,15.00,95,100,1.88,NO,1600841478
464,puerto del rosario,28.50,-13.86,24.00,57,64,2.60,ES,1600841478
465,reftinskiy,57.09,61.68,9.00,70,75,12.00,RU,1600841479
466,zhigansk,66.77,123.37,11.45,48,35,0.88,RU,1600841479


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
!jupyter nbextension enable --py --sys-prefix widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [5]:
gmaps.configure(api_key=g_key)

In [6]:
# Access maps with unique API key

locations=city_weather[["Lat","Lng"]]
weights=city_weather['Humidity']

In [7]:
fig=gmaps.figure()

heatlayer=gmaps.heatmap_layer(
    locations,
    weights=weights,
    dissipating=False,
    max_intensity=100,
    point_radius=4
)
fig.add_layer(heatlayer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [12]:
good_weather_df = city_weather.loc[(city_weather['Max Temp'] < 20) & (city_weather['Cloudiness'] < 5)& (city_weather['Wind Speed'] <2)]
good_weather_df.dropna(inplace=True)
good_weather_df

C:\Users\gu202\anaconda3\envs\pythonMain\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
29,clyde river,70.47,-68.59,-6.00,79,1,1.00,CA,1600841426
164,fastiv,50.08,29.92,16.11,71,0,1.00,UA,1600841442
214,kysyl-syr,63.90,122.76,14.18,43,0,1.63,RU,1600841449
241,calbuco,-41.77,-73.13,4.00,100,0,1.00,CL,1600841452
281,krosno,49.69,21.77,16.11,73,0,0.45,PL,1600841456
294,dale,38.17,-86.99,14.00,93,1,1.42,US,1600841458
321,saldanha,-33.01,17.94,11.00,81,0,1.50,ZA,1600841462
390,myrtle beach,33.69,-78.89,14.00,87,1,1.81,US,1600841470
427,hermitage,41.23,-80.45,11.00,81,1,1.50,US,1600841401
453,dillon,34.42,-79.37,11.11,100,1,1.51,US,1600841477


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [13]:
hotel_df=good_weather_df
hotel_df['Hotel Name'] = ''
hotel_df.head()

C:\Users\gu202\anaconda3\envs\pythonMain\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
29,clyde river,70.47,-68.59,-6.00,79,1,1.00,CA,1600841426,
164,fastiv,50.08,29.92,16.11,71,0,1.00,UA,1600841442,
214,kysyl-syr,63.90,122.76,14.18,43,0,1.63,RU,1600841449,
241,calbuco,-41.77,-73.13,4.00,100,0,1.00,CL,1600841452,
281,krosno,49.69,21.77,16.11,73,0,0.45,PL,1600841456,


In [15]:
# create results dataframe
results_df=pd.DataFrame({"City":hotel_df["City"],
                      "Country":hotel_df["Country"],
                      "Hotel Name": "",
                      "Latitude": "",
                      "Longtitude": ""
                     })
results_df

,City,Country,Hotel Name,Latitude,Longtitude
29,clyde river,CA,,,
164,fastiv,UA,,,
214,kysyl-syr,RU,,,
241,calbuco,CL,,,
281,krosno,PL,,,
294,dale,US,,,
321,saldanha,ZA,,,
390,myrtle beach,US,,,
427,hermitage,US,,,
453,dillon,US,,,


In [ ]:
# Set parameters to search for hotels with 5000 meters.
# Hit the Google Places API for each city's coordinates.
# Store the first Hotel result into the DataFrame.

# change the type for lat and lng to string

locations["Lat"]=locations["Lat"].astype(str)
locations["Lng"]=locations["Lng"].astype(str)

for i,row in locations.iterrows():
    try:
        target_search="Hotel"
        target_radius="5000"
        target_coordinates=row["Lat"]+','+row["Lng"]
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        params = {
            "location": target_coordinates,
            "keyword": target_search,
            "radius": target_radius,
            "key": g_key
        }
        response = requests.get(base_url, params=params).json()
        lat = response["results"][0]["geometry"]["location"]["lat"]
        lng = response["results"][0]["geometry"]["location"]["lng"]
        name= response["results"][0]["name"]

        results_df.loc[i,'Latitude']=lat
        results_df.loc[i,'Longtitude']=lng
        results_df.loc[i,'Hotel Name']=name
        hotel_df.loc[i,'Hotel Name']=name
    except:
        results_df.loc[i,'Hotel Name']="NA"
        hotel_df.loc[i,'Hotel Name']="NA"
        
hotel_df.dropna(inplace=True)
results_df.dropna(inplace=True)
hotel_df

C:\Users\gu202\anaconda3\envs\pythonMain\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\gu202\anaconda3\envs\pythonMain\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\gu202\anaconda3\envs\pythonMain\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org

In [65]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [66]:

# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(markers)
fig

# Display figure

Figure(layout=FigureLayout(height='420px'))